In [1]:
%cd ../

/home/users/dmoreno2016/VT_Model_for_LightCurves_Classification


/home/users/dmoreno2016/miniconda3/envs/visualtrans_git/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import shutil
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("file:./results/ml-runs")

In [3]:
def get_mlflow_runs(experiment_id):
    """ Obtener los IDs de las corridas activas de MLflow para un experimento específico. """
    client = MlflowClient()
    runs = client.search_runs(experiment_ids=[experiment_id])
    return {run.info.run_id for run in runs}

def get_filesystem_runs(directory):
    """ Obtener los IDs de las corridas desde los directorios del sistema de archivos. """
    return {run_id for run_id in os.listdir(directory) if os.path.isdir(os.path.join(directory, run_id))}

def main(mlflow_directory, experiment_id):
    """ Compara los IDs de las corridas en MLflow y en el sistema de archivos, eliminando las huérfanas. """
    mlflow_runs = get_mlflow_runs(experiment_id)
    filesystem_runs = get_filesystem_runs(mlflow_directory)
    
    # Encuentra corridas en el sistema de archivos que no están en MLflow
    orphaned_runs = filesystem_runs - mlflow_runs
    
    if orphaned_runs:
        print("Corridas huérfanas encontradas y eliminadas:")
        for run_id in orphaned_runs:
            run_path = os.path.join(mlflow_directory, run_id)
            print(f"Eliminando: {run_path}")
            shutil.rmtree(run_path)  # Elimina el directorio y su contenido
    else:
        print("No se encontraron corridas huérfanas.")

# Configuración de MLflow
data_name = 'macho'
experiment_name = f"ft_classification/{data_name}/best_params"

experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment:
    experiment_id = experiment.experiment_id
    print(f"Experiment ID for '{experiment_name}': {experiment_id}")

    mlflow_directory = f"results/ml-runs/{experiment_id}" 
    if os.path.exists(mlflow_directory):  # Verificar que la carpeta existe antes de continuar
        main(mlflow_directory, experiment_id)
    else:
        print(f"La carpeta {mlflow_directory} no existe.")
else:
    print(f"Experiment '{experiment_name}' not found.")

Experiment ID for 'ft_classification/macho/best_params': 197991169753779357
Corridas huérfanas encontradas y eliminadas:
Eliminando: results/ml-runs/197991169753779357/e4a1eecdfdfc496e9ffc3f216a802f3a
Eliminando: results/ml-runs/197991169753779357/064921d3d979487189ff47fcc03e9224
Eliminando: results/ml-runs/197991169753779357/8d61e4e97d4544bfb3ad7cdd1cb9d455
Eliminando: results/ml-runs/197991169753779357/ba04abd8fdc949f4a9c5f7fe3eebdfe2
